In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai as genai
from IPython.display import Markdown, display, update_display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
openai = OpenAI()

claude = anthropic.Anthropic()

genai.configure()

In [ ]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-2.0-flash-exp"

gpt_system = "You are a chatbot who is sarcastic; \
you have your speculations about anything in the conversation and you challenge everything in funny way.\
You have to be a part of a group discussion and put forward your points about the topic\
full-stack developers vs specialised developer. Keep your points short and precise."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting.You have to be a part of a group discussion and put forward your points\
about the topic full-stack developers vs specialised developer. Keep your points short and precise."

gemini_system = "You are a very rational thinker and don't like beating around the bush about the topic of discussion.\
You have to be a part of a group discussion and put forward your points\
about the topic full-stack developers vs specialised developer\
Keep your points short and precise."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["Hello to all"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "assistant", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages,
        max_tokens=500  # Add max_tokens to meet API requirement
    )
    return completion.choices[0].message.content

# Function to call Claude
def call_claude():
    messages = []
    for gpt, claude_message,gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "assistant", "content": gemini})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        max_tokens=500,
        messages=messages
    )
    return message.content[0].text

# Function to call Gemini
def call_gemini():
    # Create the Gemini model instance
    gemini_model_instance = genai.GenerativeModel(
        model_name=gemini_model,  # Specify the model name here
        system_instruction=gemini_system  # Provide the system instruction
    )
    
    # Prepare conversation history with separate names to avoid overwriting
    gemini_messages_combined = []
    for gpt, claude, gemini_msg in zip(gpt_messages, claude_messages, gemini_messages):
        gemini_messages_combined.append({"role": "assistant", "content": gpt})
        gemini_messages_combined.append({"role": "user", "content": claude})
        gemini_messages_combined.append({"role": "assistant", "content": gemini_msg})
    
    # Generate content based on the conversation history
    gemini_response = gemini_model_instance.generate_content("".join([msg["content"] for msg in gemini_messages_combined]))
    
    return gemini_response.text

# Initial print
print(f"Gemini:\n{gemini_messages[0]}\n")
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

# Main loop to generate conversation
for i in range(3):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)
    
    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)